<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Test-single-video-with-rekall" data-toc-modified-id="Test-single-video-with-rekall-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test single video with rekall</a></span></li><li><span><a href="#Test-single-video-with-haotian's-algorithm" data-toc-modified-id="Test-single-video-with-haotian's-algorithm-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Test single video with haotian's algorithm</a></span></li><li><span><a href="#Test-videos" data-toc-modified-id="Test-videos-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test videos</a></span></li><li><span><a href="#Process-labeled-commercial-groundtruth" data-toc-modified-id="Process-labeled-commercial-groundtruth-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Process labeled commercial groundtruth</a></span></li></ul></div>

In [ ]:
import os
import pickle
from query.models import Video
from esper.prelude import esper_widget
from rekall.interval_list import IntervalList
from esper.rekall import *
from esper.commercial_detect import detect_commercial, visualize_commercial
from esper.commercial_detect_rekall import detect_commercial_rekall

get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')

# Load data

In [ ]:
# load data
black_frame_dict = pickle.load(open('/app/data/black_frame_all.pkl', 'rb'))
additional_field = pickle.load(open('/app/data/addtional_field.pkl', 'rb'))
commercial_gt = pickle.load(open('/app/data/commercial_gt.pkl', 'rb'))

In [ ]:
videos = Video.objects.filter(threeyears_dataset=True).all()

In [ ]:
video = Video.objects.filter(path__contains='CNNW_20110513_150000_CNN_Newsroom')[0]
# video = Video.objects.filter(id=62068)[0]
video_name = video.item_name()

# Test single video with rekall

In [ ]:
result = detect_commercial_rekall(video, '/app/data/subs/aligned/'+video_name+'.word.srt', blackframe_list=black_frame_dict[video.id], debug=True, verbose=False)

In [ ]:
# Debug commercial blocks
def intrvllists_time2fid(intrvllists, fps):
    return IntervalList([(i.start*video.fps, i.end*video.fps, 0) for i in intrvllists.get_intervals()])

commercial_intrvllists = {}
commercial_intrvllists = intrvllists_to_result({video.id: intrvllists_time2fid(result['black'], video.fps)}, color='black')
add_intrvllists_to_result(commercial_intrvllists,
                         {video.id: intrvllists_time2fid(result['arrow'], video.fps)},
                          color='black')
add_intrvllists_to_result(commercial_intrvllists,
                         {video.id: intrvllists_time2fid(result['commercials_raw'], video.fps)},
                          color='orange')
add_intrvllists_to_result(commercial_intrvllists,
                         {video.id: intrvllists_time2fid(result['lowercase'], video.fps)},
                          color='gray')
add_intrvllists_to_result(commercial_intrvllists,
                         {video.id: intrvllists_time2fid(result['blank'], video.fps)},
                          color='blue')
add_intrvllists_to_result(commercial_intrvllists,
                         {video.id: intrvllists_time2fid(result['commercials'], video.fps)},
                          color='red')
# add_intrvllists_to_result(commercial_intrvllists,
#                          {video.id: IntervalList([(start*video.fps, end*video.fps, 0) for (start, end) in commercial_gt['10y'][video.id]]) },
#                           color='green')
esper_widget(commercial_intrvllists)

# Test single video with haotian's algorithm

In [ ]:
video_desp = {'fps': video.fps, 'frame_w': video.width, 'frame_h': video.height, 'video_length': video.num_frames/video.fps, 'video_frames': video.num_frames}
commercial_list = detect_commercial(video_desp, '/app/data/subs10/'+video_name, blackframe_list=black_frame_dict[video_name])
commercial_list

# Test videos  

In [ ]:
com_dict_us, com_dict_gt = {}, {}
for video_id in sorted(commercial_gt['all']):
    com_gt = commercial_gt['all'][video_id]
    video = Video.objects.filter(id=video_id)[0]
    video_name = video.item_name()
    transcript_path = "/app/data/subs/aligned/" + video_name + '.word.srt'
    print(video_name)
    if not video_id in black_frame_dict or not os.path.exists(transcript_path):
        continue
#     video_desp = {'fps': video.fps, 'frame_w': video.width, 'frame_h': video.height, 'video_length': video.num_frames/video.fps, 'video_frames': video.num_frames}
#     commercial_list = detect_commercial(video_desp, '/app/data/subs10/'+video_name, blackframe_list=black_frame_dict[video_name], verbose=False)    
    com_us = detect_commercial_rekall(video, transcript_path, blackframe_list=black_frame_dict[video_id], debug=False)    
    com_dict_us[video_name] = com_us
    com_dict_gt[video_name] = com_gt

In [ ]:
visualize_commercial(com_dict_gt, com_dict_us)

In [ ]:
commercial_dict = pickle.load(open('/app/result/commercial/final_dict.pkl', 'rb'))

In [ ]:
# Display commercial blocksf

# commercial_dict = {video_fname: commercial_list}
commercial_intrvllists = {}
video_ids = []
for video_id in sorted(commercial_dict):
    video = Video.objects.filter(id=video_id)[0]
    video_ids.append(video.id)
    commercial_intrvllists[video.id] = IntervalList([(start*video.fps, end*video.fps, 0) for (start, end) in commercial_dict[video_id]])
esper_widget(intrvllists_to_result(commercial_intrvllists))

# Process labeled commercial groundtruth

In [ ]:
commercial_gt = pickle.load(open('/app/data/commercial_gt.pkl', 'rb'))

In [ ]:
from esper.util import time2second
com_gt = {}
for line in open('/app/data/commercial_gt4.csv'):
    columns = line[:-1].split(',')
    video_id = int(columns[0])
    video = Video.objects.filter(id=video_id)[0]
#     video = Video.objects.filter(path__contains=columns[0])[0]
#     video_id = video.id
    
    duration = int(video.num_frames / video.fps)
    com_gt[video_id] = []
#     print(video_id)
    for i in range(1, len(columns)):
        if columns[i] == '':
            continue
        span = columns[i].split('-')
        start = span[0].split(':')
        end = span[1].split(':')
        start = time2second((int(start[0]), int(start[1]), int(start[2])))
        end = time2second((int(end[0]), int(end[1]), int(end[2])))
        com_gt[video_id].append((start, end))
com_gt

In [ ]:
# commercial_gt['10y'] = com_gt
commercial_gt['all'] = {**commercial_gt['all'], **commercial_gt['10y']}

In [ ]:
pickle.dump(commercial_gt, open('/app/data/commercial_gt.pkl', 'wb'))